In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding, Input, RepeatVector
from keras.optimizers import SGD

2023-07-22 15:39:28.562015: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
true = pd.read_csv('True.csv')
fake = pd.read_csv('Fake.csv')
true['label'] = 1
fake['label'] = 0
frames = [true.loc[:10000][:], fake.loc[:10000][:]]  #Edit this to control the size of dataset
df = pd.concat(frames)
df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [3]:
X = df.drop('label', axis=1) 
y = df['label']
df = df.dropna()
df2 = df.copy()
df2 = df2.reset_index(drop = True)


In [4]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(df2)):
    review = re.sub('[^a-zA-Z]', ' ', df2['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X = tfidf_v.fit_transform(corpus).toarray()
y = df2['label']

In [6]:
def baseline_model() : 
    model = Sequential()
    model.add(Dense(256, activation = 'relu', kernel_initializer = 'normal'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation = 'relu', kernel_initializer = 'normal'))
    model.add(Dropout(0.5))
    model.add(Dense(80, activation = 'relu', kernel_initializer = 'normal'))
    model.add(Dense(1, activation = 'sigmoid', kernel_initializer = 'normal'))
    
    sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return model

    

In [7]:
model = baseline_model()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

model.fit(X_train, y_train, epochs=30, batch_size=32)
results = model.evaluate(X_test, y_test)

print(model.predict(X_test))
print(results)


Epoch 1/30
501/501 [==============================] - 8s 13ms/step - loss: 0.0921 - accuracy: 0.9585
Epoch 2/30
501/501 [==============================] - 7s 14ms/step - loss: 0.0088 - accuracy: 0.9974
Epoch 3/30
501/501 [==============================] - 7s 14ms/step - loss: 0.0042 - accuracy: 0.9987
Epoch 4/30
501/501 [==============================] - 7s 14ms/step - loss: 0.0014 - accuracy: 0.9995
Epoch 5/30
501/501 [==============================] - 7s 14ms/step - loss: 0.0017 - accuracy: 0.9994
Epoch 6/30
501/501 [==============================] - 6s 13ms/step - loss: 5.8413e-04 - accuracy: 0.9998
Epoch 7/30
501/501 [==============================] - 6s 13ms/step - loss: 0.0024 - accuracy: 0.9993
Epoch 8/30
501/501 [==============================] - 6s 13ms/step - loss: 0.0012 - accuracy: 0.9996
Epoch 9/30
501/501 [==============================] - 7s 13ms/step - loss: 8.7396e-04 - accuracy: 0.9998
Epoch 10/30
501/501 [==============================] - 6s 13ms/step - loss: 4.2067e

In [9]:
import pickle 
from keras.models import load_model

vectorname = 'vectorizer.pkl'
pickle.dump(tfidf_v, open(vectorname, 'wb'))
model.save('nn_model.keras')